MC-DC-GAN (Multi Class Deep Convulution Generative Adversarial Networks )

by Ayush Agarwal , Electronics Engineering , IIT BHU Varanasi 

A techniques invented by me , to make the GAN draw 2 different classes of drawings as per the input key given to it .

The main code

In [27]:
#MCDCGAN
# By Ayush Agarwal , Electronics Engineering , IIT BHU Varanasi 

#The main code
##############################################################################################
# base image size would be 8x8 , as I don't have that much time and computational resources 
#importing the libraries
import numpy as np 
import pandas as pd 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from functools import partial
import matplotlib.pyplot as plt
########################################################################
#making the image 
#img1 = (np.eye(8)*255)
img1 = np.ones(shape=(8,8))
img2 = np.ones(shape=(8,8))
# for i in range(0,8):
#     for j in range(0,8,2):
#         img1[i,j]=255
# for j in range(0,8):
#     for i in range(0,8,2):
#         img2[i,j]=255
img1[:][2:4]=255
img2[:][4:6]=255
#img1[:][3]=255
#img2[4:8][:]=255
#img2[5][:]=255
# img1[1:4][1:4]=255
# img2[5:8][5:8]=255
######################################
# printing the image
print()
print(img1)
print()
print(img2)
#################################################
img1 = tf.reshape(img1, (-1, 8, 8, 1))
img2 = tf.reshape(img2, (-1, 8, 8, 1))
# seed1 = [[1,0],[0,1]]
# seed1 = tf.constant([[1,0,0]])
# seed2 = tf.constant([[0,1], [1,0]])
# the outputs of the discriminator I named seed 
seed1=tf.constant([[1,0,0]])
seed2=tf.constant([[0,1,0]])
imgs = tf.convert_to_tensor(np.array([img1,img2]))
imgs = tf.reshape(imgs, (-1, 8, 8, 1))
seeds = tf.constant([[1,0,0],[0,1,0]])
#############################################################################################
# The Discriminator model 
Discriminator = keras.Sequential([
    
    layers.Conv2D(filters=25, kernel_size=2,# activation="relu", #padding='same',
                  input_shape=[8, 8, 1]),
    layers.Conv2D(filters=50, kernel_size=2, activation=partial(tf.nn.leaky_relu, alpha=0.01)#"relu"#, padding='same'
                 ),
    layers.Conv2D(filters=75, kernel_size=2, activation=partial(tf.nn.leaky_relu, alpha=0.01)#"relu"#, padding='same'
                 ),
    layers.Conv2D(filters=50, kernel_size=2, activation=partial(tf.nn.leaky_relu, alpha=0.01)#"relu"#, padding='same'
                 ),
    layers.Conv2D(filters=25, kernel_size=2, activation=partial(tf.nn.leaky_relu, alpha=0.01)#"relu"#, padding='same'
                 ),
    layers.Conv2D(filters=1, kernel_size=2, activation=partial(tf.nn.leaky_relu, alpha=0.01)#"relu"#, padding='same'
                 ),
    layers.Flatten(),
    layers.Dense(3, activation='softmax'),
])
# was facing the dying relu problem so used a leakyrelu here 
Discriminator.summary()
Discriminator.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# training the discriminator 
early_stopping = keras.callbacks.EarlyStopping(
    patience=10,
    min_delta=0.01,
    restore_best_weights=True,
)

history = Discriminator.fit(
    imgs,seeds,
    validation_data=(imgs,seeds),
    epochs=25,
    callbacks=[early_stopping]
)

history_df = pd.DataFrame(history.history)
history_df.loc[1:, ['loss', 'val_loss']].plot()
history_df.loc[1:, ['accuracy', 'val_accuracy']].plot()
plt.show()

prediction1 = Discriminator.predict(img1)
prediction2 = Discriminator.predict(img2)
print("Prediction on class 1 by discriminator :")
print(prediction1)
print()
print("Prediction on class 2 by discriminator :")
print(prediction2)
#####################################################################################
# Making the Generator 

Generator = keras.Sequential([
    layers.Dense(4, use_bias=False, input_shape=(4,)),
    layers.Reshape((2, 2,1)),
  #  layers.Conv2DTranspose(25, (2, 2), strides=(1, 1), use_bias=False),# padding='same',
    layers.Conv2DTranspose(500, (3, 3), strides=(1, 1), use_bias=False, activation=partial(tf.nn.leaky_relu, alpha=0.01)),# padding='same',
    layers.AveragePooling2D(pool_size=(2, 2), strides=(1, 1), padding='same'),
    layers.Conv2DTranspose(1000, (1, 1), strides=(1, 1), use_bias=False, activation=partial(tf.nn.leaky_relu, alpha=0.01)),# padding='same',
   # layers.Dropout(0.3),
    layers.Conv2DTranspose(500, (3, 3), strides=(1, 1), use_bias=False, activation=partial(tf.nn.leaky_relu, alpha=0.01)),# padding='same',
   # layers.Conv2DTranspose(200, (2, 2), strides=(1, 1), use_bias=False, activation=partial(tf.nn.leaky_relu, alpha=0.01)),# padding='same',
    layers.AveragePooling2D(pool_size=(2, 2), strides=(1, 1), padding='same'),
   # layers.Dropout(0.3),
    layers.Conv2DTranspose(500, (2, 2), strides=(1, 1), use_bias=False, activation=partial(tf.nn.leaky_relu, alpha=0.01)),# padding='same',
    layers.Conv2DTranspose(1, (2, 2), strides=(1, 1), use_bias=False),# padding='same',
    
])
Generator.summary()
# brush1=tf.constant([[1,0,0,0]])
# brush2=tf.constant([[0,0,0,1]])
brushes = tf.constant([[255,1,255,1],[1,255,1,255]])
print("Generator output when it wasnt trained :")
untrained_prediction1 = Generator.predict(brush1)
plt.imshow(untrained_prediction1[0][:][:][:], cmap='gray')
#print(prediction1)
################################################################
#Making the GAN 
Discriminator.trainable = False
#GAN = tf.keras.layers.Concatenate()([Generator, Discriminator])
#GAN = tf.keras.Model(inputs=Generator.outputs, outputs=Discriminator.outputs)
GAN = keras.Sequential([
    Generator ,
    Discriminator
])
    
GAN.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

########################################################################
# Training the GAN 
early_stopping = keras.callbacks.EarlyStopping(
    patience=10,
    min_delta=0.01,
    restore_best_weights=True,
)

history2 = GAN.fit(
    brushes,seeds,
    validation_data=(brushes,seeds),
    epochs=100,
    callbacks=[early_stopping]
)

history_df2 = pd.DataFrame(history2.history)
history_df2.loc[1:, ['loss', 'val_loss']].plot()
history_df2.loc[1:, ['accuracy', 'val_accuracy']].plot()
plt.show()      

##############################################################
print()
print("Painting using class 1 ( brush 1 )")
drawing1 = Generator.predict(brush1)
plt.imshow(drawing1[0][:][:][:], cmap='gray')
plt.show()
print("Painting using class 2 ( brush 2 )")
drawing2 = Generator.predict(brush2)
plt.imshow(drawing2[0][:][:][:], cmap='gray')
plt.show()

######################################################################

As we can see , the white part is a bit up in the first image , whereas it is a bit below in the second image . These 2 different patterns it has absorbed during training from the 2 different images it was trained on . And more importantly , I could make these 2 drawings not randomly but by my own choice by using different inputs to the generator (which I have called brushes) . Since I have used only 1 picture of each class , it has absorbed very little . But I feel that when lots of images would be fed into the GAN and differently labelled , (say for eg one picture is in warm colours and other in cool colours ) , then I can select different modes/keys/(brushes as per my project ) and make different types of images from the GAN as per my own choice . This is all I wanted to show in my project . The results weren't personally satisfactory to me since I was expecting the GAN to give out perfect pictures , but alas , the dataset isn't big enough . Noteworthy it is that the Discriminator is quite well trained yet the Generator isn't.


Rest of the file is my rough work , feel free to go throught it . 

generator code work here

In [141]:
import numpy as np 
import pandas as pd 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from functools import partial
import matplotlib.pyplot as plt

Generator = keras.Sequential([
    layers.Dense(4, use_bias=False, input_shape=(4,)),
    layers.Reshape((2, 2,1)),
    layers.Conv2DTranspose(25, (2, 2), strides=(1, 1), use_bias=False),# padding='same',
    layers.Conv2DTranspose(50, (2, 2), strides=(1, 1), use_bias=False),# padding='same',
    layers.Conv2DTranspose(75, (2, 2), strides=(1, 1), use_bias=False),# padding='same',
    layers.Conv2DTranspose(50, (2, 2), strides=(1, 1), use_bias=False),# padding='same',
    layers.Conv2DTranspose(50, (2, 2), strides=(1, 1), use_bias=False),# padding='same',
    layers.Conv2DTranspose(1, (2, 2), strides=(1, 1), use_bias=False),# padding='same',
    
])
Generator.summary()
brush1=tf.constant([[1,0,0,0]])
untrained_prediction1 = Generator.predict(brush1)

plt.imshow(untrained_prediction1[0][:][:][:], cmap='gray')
#print(prediction1)

Perfectly working code for discriminator 

do 

not 

change

In [97]:
# perfectly working code for a discriminator between horizontal and vertical image lines 

# base image size would be 8x8 , as I don't have that much time and computational resources 

import numpy as np 
import pandas as pd 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from functools import partial
import matplotlib.pyplot as plt
import pickle
import dill
#from sklearn.externals import joblib

img1 = np.zeros(shape=(8,8))
img2 = np.zeros(shape=(8,8))
for i in range(0,8):
    for j in range(0,8,2):
        img1[i,j]=255
for j in range(0,8):
    for i in range(0,8,2):
        img2[i,j]=255
print()
print(img1)
print()
print(img2)
plt.imshow(img1, cmap='gray')
plt.show()
plt.imshow(img2, cmap='gray')
plt.show()
img1 = tf.reshape(img1, (-1, 8, 8, 1))
img2 = tf.reshape(img2, (-1, 8, 8, 1))
# seed1 = [[1,0],[0,1]]
# seed1 = tf.constant([[1,0,0]])
# seed2 = tf.constant([[0,1], [1,0]])
seed1=tf.constant([[1,0,0]])
seed2=tf.constant([[0,1,0]])
imgs = tf.convert_to_tensor(np.array([img1,img2]))
imgs = tf.reshape(imgs, (-1, 8, 8, 1))
#imgs = tf.expand_dims(imgs, 1) 
#seeds=tf.convert_to_tensor(([seed1,seed2]))
seeds = tf.constant([[1,0,0],[0,1,0]])

Discriminator = keras.Sequential([
    
    layers.Conv2D(filters=25, kernel_size=2,# activation="relu", #padding='same',
                  input_shape=[8, 8, 1]),
    layers.Conv2D(filters=25, kernel_size=2, activation=partial(tf.nn.leaky_relu, alpha=0.01)#"relu"#, padding='same'
                 ),
    layers.Conv2D(filters=25, kernel_size=2, activation=partial(tf.nn.leaky_relu, alpha=0.01)#"relu"#, padding='same'
                 ),
    layers.Conv2D(filters=25, kernel_size=2, activation=partial(tf.nn.leaky_relu, alpha=0.01)#"relu"#, padding='same'
                 ),
    layers.Conv2D(filters=25, kernel_size=2, activation=partial(tf.nn.leaky_relu, alpha=0.01)#"relu"#, padding='same'
                 ),
    layers.Conv2D(filters=1, kernel_size=2, activation=partial(tf.nn.leaky_relu, alpha=0.01)#"relu"#, padding='same'
                 ),
    layers.Flatten(),
    layers.Dense(3, activation='softmax'),
])
# was facing the dying relu problem so used a leakyrelu here 
Discriminator.summary()

#optimizer = tf.keras.optimizers.Adam(epsilon=0.01)
# Discriminator.compile(
#     #optimizer=optimizer,
#     optimizer='adam',
#     #loss='mae',
#     loss='sparse_categorical_crossentropy',
#     metrics=['accuracy'],
# )

Discriminator.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


early_stopping = keras.callbacks.EarlyStopping(
    patience=10,
    min_delta=0.01,
    restore_best_weights=True,
)

history = Discriminator.fit(
    imgs,seeds,
    validation_data=(imgs,seeds),
    epochs=25,
    callbacks=[early_stopping]
)

history_df = pd.DataFrame(history.history)
history_df.loc[1:, ['loss', 'val_loss']].plot()
history_df.loc[1:, ['accuracy', 'val_accuracy']].plot()

prediction1 = Discriminator.predict(img1)
prediction2 = Discriminator.predict(img2)
print(prediction1)
print()
print(prediction2)
#tf.saved_model.save(Discriminator, mobilenet_save_path)
tf.saved_model.save(Discriminator,'./')
#Discriminator.save('Discriminator1.h5')
#joblib.dump(Discriminator, 'Discriminator2.pkl')
# dill.dump(Discriminator, open('Discriminator1.pkl', 'wb'))
#pickle.dump(Discriminator, open('Discriminator2.pkl','wb'))














In [80]:
pip install joblib

In [100]:
import numpy as np 
import pandas as pd 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from functools import partial
import matplotlib.pyplot as plt
import pickle
import dill

from keras.models import load_model

import keras

Discriminator = keras.Sequential([
    
    layers.Conv2D(filters=25, kernel_size=2,# activation="relu", #padding='same',
                  input_shape=[8, 8, 1]),
    layers.Conv2D(filters=25, kernel_size=2, activation=partial(tf.nn.leaky_relu, alpha=0.01)#"relu"#, padding='same'
                 ),
    layers.Conv2D(filters=25, kernel_size=2, activation=partial(tf.nn.leaky_relu, alpha=0.01)#"relu"#, padding='same'
                 ),
    layers.Conv2D(filters=25, kernel_size=2, activation=partial(tf.nn.leaky_relu, alpha=0.01)#"relu"#, padding='same'
                 ),
    layers.Conv2D(filters=25, kernel_size=2, activation=partial(tf.nn.leaky_relu, alpha=0.01)#"relu"#, padding='same'
                 ),
    layers.Conv2D(filters=1, kernel_size=2, activation=partial(tf.nn.leaky_relu, alpha=0.01)#"relu"#, padding='same'
                 ),
    layers.Flatten(),
    layers.Dense(3, activation='softmax'),
])
#Discriminator= tf.saved_model.load('../input/discriminator4/saved_model (1).pb')
#loaded = tf.saved_model.load(mobilenet_save_path)
#new_model.load_weights('CIFAR1006.h5')
#Discriminator.load_weights('./Discriminator1.pkl')
#model1 = load_model('./Discriminator1.h5')
img1 = np.zeros(shape=(8,8))
img2 = np.zeros(shape=(8,8))
for i in range(0,8):
    for j in range(0,8,2):
        img1[i,j]=255
for j in range(0,8):
    for i in range(0,8,2):
        img2[i,j]=255
        
y_pred1 = Discriminator.predict(img1)
y_pred2 = Discriminator.predict(img2)
print(y_pred1)
print(y_pred2)